In [1]:
%autosave 60

Autosaving every 60 seconds


In [12]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils,datasets
from tqdm.notebook import tqdm
import torchvision
from torch import nn
import os
import pickle
import sys
from argparse import ArgumentParser

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import tqdm
from torch.nn import functional as fnn
from torch.utils.data import DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from utils import NUM_PTS, CROP_SIZE
from utils import ScaleMinSideToSize, CropCenter, TransformByKeys
from utils import ThousandLandmarksDataset
from utils import restore_landmarks_batch, create_submission
import cv2
from torch.utils import data
import albumentations as A


torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:

TRAIN_SIZE = 0.8
NUM_PTS = 971
CROP_SIZE = 128
SUBMISSION_HEADER = "file_name,Point_M0_X,Point_M0_Y,Point_M1_X,Point_M1_Y,Point_M2_X,Point_M2_Y,Point_M3_X,Point_M3_Y,Point_M4_X,Point_M4_Y,Point_M5_X,Point_M5_Y,Point_M6_X,Point_M6_Y,Point_M7_X,Point_M7_Y,Point_M8_X,Point_M8_Y,Point_M9_X,Point_M9_Y,Point_M10_X,Point_M10_Y,Point_M11_X,Point_M11_Y,Point_M12_X,Point_M12_Y,Point_M13_X,Point_M13_Y,Point_M14_X,Point_M14_Y,Point_M15_X,Point_M15_Y,Point_M16_X,Point_M16_Y,Point_M17_X,Point_M17_Y,Point_M18_X,Point_M18_Y,Point_M19_X,Point_M19_Y,Point_M20_X,Point_M20_Y,Point_M21_X,Point_M21_Y,Point_M22_X,Point_M22_Y,Point_M23_X,Point_M23_Y,Point_M24_X,Point_M24_Y,Point_M25_X,Point_M25_Y,Point_M26_X,Point_M26_Y,Point_M27_X,Point_M27_Y,Point_M28_X,Point_M28_Y,Point_M29_X,Point_M29_Y\n"


In [4]:
PATH_TRAIN='data_clean'


In [119]:
class MyAssDataset(data.Dataset):
    def __init__(self,images_paths,landmarks_path,preprocessing, transforms):
        super(MyAssDataset, self).__init__()
        
        landmark_file_name = landmarks_path
        

        self.image_paths = images_paths
        self.image_names = [os.path.split(path)[-1] for path in self.image_paths]
        self.landmarks = []

        with open(landmark_file_name, "rt") as fp:
            num_lines = sum(1 for line in fp)
        num_lines -= 1  # header

        with open(landmark_file_name, "rt") as fp:
            for i, line in tqdm.tqdm(enumerate(fp), total=num_lines + 1):
                if i == 0:
                    continue  # skip header
                
                elements = line.strip().split("\t")
                
                if elements[0] in self.image_names:
                    
                    landmarks = list(map(np.int, elements[1:]))
                    landmarks = np.array(landmarks, dtype=np.int).reshape((len(landmarks) // 2, 2))
                    self.landmarks.append(landmarks)   
                
        self.landmarks = torch.as_tensor(self.landmarks)
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __getitem__(self, idx):
        sample = {}
        
        landmarks = self.landmarks[idx]
        sample["landmarks"] = landmarks

        image = cv2.imread(self.image_paths[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        sample["image"] = image

        if self.preprocessing is not None:
            sample = self.preprocessing(sample)
            
        sample['image'] = np.array(sample['image'])
        if self.transforms is not None:
            sample = self.transforms(image=sample['image'], keypoints=sample['landmarks'].numpy().reshape(-1,2))

        return sample

    def __len__(self):
        return len(self.image_names)

In [120]:
def show_landmarks(image, landmarks):
    """Show image with landmarks"""
    plt.imshow(image.permute(1,2,0))
    
    plt.scatter(landmarks.reshape(-1,2)[:, 0], landmarks.reshape(-1,2)[:, 1], s=10, marker='.', c='r')
    plt.pause(0.001)

In [121]:
preprocessing = transforms.Compose([
    ScaleMinSideToSize((CROP_SIZE, CROP_SIZE)),
    CropCenter(CROP_SIZE),
    TransformByKeys(transforms.ToPILImage(), ("image",)),
    TransformByKeys(transforms.ToTensor(), ("image",)),
#     TransformByKeys(transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25]), ("image",)),
     
])

In [122]:
from albumentations.pytorch import ToTensor

In [123]:
train_transform = A.Compose([
        
        A.HorizontalFlip(p=0.5),
        A.OneOf([
            A.HueSaturationValue(p=0.5), 
            A.RGBShift(p=0.7)
        ], p=1),                          
        A.RandomBrightnessContrast(p=0.5),
    ToTensor()
    
    ], 
    keypoint_params=A.KeypointParams(format='xy'),
)

In [124]:
all_image_paths = [os.path.join(PATH_TRAIN,'images',name) for name in os.listdir(os.path.join(PATH_TRAIN,'images'))]

In [125]:
train_images,val_images=train_test_split(all_image_paths,test_size=0.33)

In [126]:
train_dataset  = MyAssDataset(train_images[:100],os.path.join(PATH_TRAIN, 'landmarks.csv'), preprocessing=preprocessing,transforms=train_transform)

100%|██████████| 59525/59525 [00:04<00:00, 14104.34it/s]


In [127]:
train_dataset[0]['image']

ValueError: Expected y for keypoint (52.321167, 96.59124, 0.0, 0) to be in the range [0.0, 3], got 96.59123992919922.

In [72]:
show_landmarks(train_dataset[0]['image'],train_dataset[0]['landmarks'])

error: OpenCV(4.5.1) /tmp/pip-req-build-jhawztrk/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [70]:
train_dataset.landmarks

tensor([[[162, 311],
         [159, 311],
         [156, 311],
         ...,
         [123, 254],
         [112, 163],
         [191, 166]],

        [[123, 210],
         [121, 210],
         [119, 211],
         ...,
         [ 95, 176],
         [ 80, 119],
         [128, 115]],

        [[109, 203],
         [107, 203],
         [105, 203],
         ...,
         [ 73, 157],
         [ 75, 107],
         [129, 106]],

        ...,

        [[ 66, 130],
         [ 65, 130],
         [ 64, 130],
         ...,
         [ 51, 103],
         [ 49,  67],
         [ 84,  68]],

        [[ 57, 126],
         [ 55, 126],
         [ 54, 125],
         ...,
         [ 46,  99],
         [ 44,  64],
         [ 78,  65]],

        [[225, 465],
         [222, 465],
         [218, 465],
         ...,
         [164, 375],
         [142, 274],
         [256, 253]]])

In [ ]:
val_dataset = MyAssDataset(val_images,os.path.join(PATH_TRAIN, 'landmarks.csv'), preprocessing=preprocessing,transforms=None)

In [ ]:

def train(model, loader, loss_fn, optimizer, device,scheduler,transforms):
    model.train()
    train_loss = []
    for batch in tqdm.tqdm(loader, total=len(loader), desc="training..."):
        images = batch["image"].to(device)  # B x 3 x CROP_SIZE x CROP_SIZE
        landmarks = batch["landmarks"]  # B x (2 * NUM_PTS)

        pred_landmarks = model(images).cpu()  # B x (2 * NUM_PTS)
        loss = loss_fn(pred_landmarks, landmarks, reduction="mean")
        train_loss.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    scheduler.step()
    return np.mean(train_loss)


def validate(model, loader, loss_fn, device):
    model.eval()
    val_loss = []
    for batch in tqdm.tqdm(loader, total=len(loader), desc="validation..."):
        images = batch["image"].to(device)
        landmarks = batch["landmarks"]

        with torch.no_grad():
            pred_landmarks = model(images).cpu()
        loss = loss_fn(pred_landmarks, landmarks, reduction="mean")
        val_loss.append(loss.item())

    return np.mean(val_loss)


def predict(model, loader, device):
    model.eval()
    predictions = np.zeros((len(loader.dataset), NUM_PTS, 2))
    for i, batch in enumerate(tqdm.tqdm(loader, total=len(loader), desc="test prediction...")):
        images = batch["image"].to(device)

        with torch.no_grad():
            pred_landmarks = model(images).cpu()
        pred_landmarks = pred_landmarks.numpy().reshape((len(pred_landmarks), NUM_PTS, 2))  # B x NUM_PTS x 2

        fs = batch["scale_coef"].numpy()  # B
        margins_x = batch["crop_margin_x"].numpy()  # B
        margins_y = batch["crop_margin_y"].numpy()  # B
        prediction = restore_landmarks_batch(pred_landmarks, fs, margins_x, margins_y)  # B x NUM_PTS x 2
        predictions[i * loader.batch_size: (i + 1) * loader.batch_size] = prediction

    return predictions

In [ ]:

def train(model, loader, loss_fn, optimizer, device,scheduler):
    model.train()
    train_loss = []
    for batch in tqdm.tqdm(loader, total=len(loader), desc="training..."):
        images = batch["image"].to(device)  # B x 3 x CROP_SIZE x CROP_SIZE
        landmarks = batch["landmarks"]  # B x (2 * NUM_PTS)

        pred_landmarks = model(images).cpu()  # B x (2 * NUM_PTS)
        loss = loss_fn(pred_landmarks, landmarks, reduction="mean")
        train_loss.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    scheduler.step()
    return np.mean(train_loss)


def validate(model, loader, loss_fn, device):
    model.eval()
    val_loss = []
    for batch in tqdm.tqdm(loader, total=len(loader), desc="validation..."):
        images = batch["image"].to(device)
        landmarks = batch["landmarks"]

        with torch.no_grad():
            pred_landmarks = model(images).cpu()
        loss = loss_fn(pred_landmarks, landmarks, reduction="mean")
        val_loss.append(loss.item())

    return np.mean(val_loss)


def predict(model, loader, device):
    model.eval()
    predictions = np.zeros((len(loader.dataset), NUM_PTS, 2))
    for i, batch in enumerate(tqdm.tqdm(loader, total=len(loader), desc="test prediction...")):
        images = batch["image"].to(device)

        with torch.no_grad():
            pred_landmarks = model(images).cpu()
        pred_landmarks = pred_landmarks.numpy().reshape((len(pred_landmarks), NUM_PTS, 2))  # B x NUM_PTS x 2

        fs = batch["scale_coef"].numpy()  # B
        margins_x = batch["crop_margin_x"].numpy()  # B
        margins_y = batch["crop_margin_y"].numpy()  # B
        prediction = restore_landmarks_batch(pred_landmarks, fs, margins_x, margins_y)  # B x NUM_PTS x 2
        predictions[i * loader.batch_size: (i + 1) * loader.batch_size] = prediction

    return predictions

In [ ]:
```
transform = A.Compose([
    A.RandomCrop(width=330, height=330),
    A.RandomBrightnessContrast(p=0.2),
    ...
], keypoint_params=A.KeypointParams(format='xy'))


# в функции train
for landmarks_batch, image_batch : dataset(не помню что тут): # возвращает данные после нормализации кропа и ресайза до 128x128

   # тут сдвиги и повороты и флипы
   landmarks_batch_transformed, image_batch_transformed =  transform(landmarks_batch, image_batch)
   
   landmarks_batch_predicted = model(image_batch_transformed)

   loss_fn(landmarks_batch_predicted, image_batch_transformed)
   ....

# в функции val
for landmarks_batch, image_batch : dataset(не помню что тут): # возвращает данные после нормализации кропа и ресайза до 128x128
   landmarks_batch_predicted = model(image_batch)
   loss_fn(landmarks_batch_predicted, landmarks_batch)
   ....


# в функции test
for image_batch : test_dataset(не помню что тут): # возвращает данные после нормализации кропа и ресайза до 128x128
   landmarks_batch_predicted = model(image_batch)